In [ ]:
!pip3 install -q --upgrade langchain langchain_community langchain-text-splitters langchain-google-genai 
!pip3 install -q --upgrade faiss-cpu google-generativeai 
!pip3 install -q python-dotenv pymupdf

## Aula 02 — RAG

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyMuPDFLoader

from pathlib import Path

from dotenv import load_dotenv
import os

In [4]:
load_dotenv()

True

In [5]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [6]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.0, 
    api_key = GOOGLE_API_KEY
    )

In [ ]:
docs = []

for n in Path("./pdfs").glob("*.pdf"): 
    try:
        loader = PyMuPDFLoader(str(n)) 
        
        docs.extend(loader.load())
        print(f"Carregado com sucesso {n.name}")
    except Exception as e:
        print(f"Erro ao carregar arquivo {n.name}: {e}")
        
print(f"Total de documentos carregados {len(docs)}")

Carregado com sucesso Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso Políticas de Home Office.pdf
Total de documentos carregados 3


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30) 

chunks = splitter.split_documents(docs)

In [15]:
chunks

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Política de Reembolsos (Viagens e \nDespesas) \n \n1.\u200b Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a \ndespesa.\u200b\n \n2.\u200b Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não \nsão reembolsáveis.\u200b'),
 Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pd

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model  = "models/gemini-embedding-001",
    google_api_key = GOOGLE_API_KEY
)

In [19]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

In [20]:
retriever = vectorstore.as_retriever(
    search_type = "similarity_score_threshold", 
    search_kwargs = {"score_threshold":0.3, "k":4}
    )

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain # Pega todos os documentos e utiliza como contexto

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm, prompt_rag)

In [ ]:
from typing import Literal, List, Dict

def perguntar_politica_rag(pergunta: str) -> Dict:
    